# 데이터 결측치 처리

## 채우기

- 채우기(fill)
  - 비어있는 값을 채움
- 일반적으로 드롭한 후에 남은 값들을 채우기 처리
- 평균, 최빈값 등 데이터의 분포를 고려해서 채움


In [1]:
import pandas as pd
import numpy as np

raw_data = {
    "first_name": ["Jason", np.nan, "Tina", "Jake", "Amy"],
    "last_name": ["Miller", np.nan, "Ali", "Milner", "Cooze"],
    "age": [42, np.nan, 36, 24, 73],
    "sex": ["m", np.nan, "f", "m", "f"],
    "preTestScore": [4, np.nan, np.nan, 2, 3],
    "postTestScore": [25, np.nan, np.nan, 62, 70],
}
df = pd.DataFrame(
    raw_data,
    columns=["first_name", "last_name", "age", "sex", "preTestScore", "postTestScore"],
)
df

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


- 함수 fillna 사용


In [ ]:
df.fillna(0)  # 모든 결측값을 숫자 0으로

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,0,0,0.0,0,0.0,0.0
2,Tina,Ali,36.0,f,0.0,0.0
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


- 채우기(fill)
  - 빈 값에 평균값을 채우려면 열 단위의 평균값을 계산하여 해당 열에만 값을 채움


In [5]:
df["preTestScore"] = df["preTestScore"].fillna(df["preTestScore"].mean())
df

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,3.0,NaN
2,Tina,Ali,36.0,f,3.0,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


- 열별 분포를 고려하여 채울 수 있음
- groupby 함수로 각 인덱스의 성별에 따라 빈칸을 채움


In [6]:
df.groupby("sex")["postTestScore"].transform("mean")

0    43.5
1     NaN
2    70.0
3    43.5
4    70.0
Name: postTestScore, dtype: float64

- fillna 함수 안에 transform을 사용하여 인덱스를 기반으로 채울 수 있음
- 일반적으로 쓰이는 기법


In [7]:
df["postTestScore"] = df["postTestScore"].fillna(
    df.groupby("sex")["postTestScore"].transform("mean")
)
df

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,3.0,NaN
2,Tina,Ali,36.0,f,3.0,70.0
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


# 원핫 인코딩

- 원핫 인코딩(one-hot encoding)
  - 범주형 데이터의 개수만큼 가변수(dummy variable)를 생성하여 존재 유무를 1 또는 0으로 표현
- color라는 변수에 {Green, Blue, Yellow} 3개의 값이 있을 때
- 3개의 가변수를 만들고 각 색상에 인덱스를 지정
  - Green의 인덱스는 0, Blue의 인덱스 1, Yellow의 인덱스는 2로 지정
- 해당 값이면 1, 아니면 0을 입력
  - {Green} -> [1,0,0]
  - {Blue} -> [0,1,0]
  - {Yellow} -> [0,0,1]


- 원핫 인코딩을 적용하려면 판다스에서 제공하는 get_dummies 함수를 이용하거나 사이킷런(scikit-learn)에서 제공하는 LabelEncoder나 OneHotEncoder를 이용


In [8]:
edges = pd.DataFrame(
    {
        "source": [0, 1, 2],
        "target": [2, 2, 3],
        "weight": [3, 4, 5],
        "color": ["red", "blue", "blue"],
    }
)
edges

,source,target,weight,color
0,0,2,3,red
1,1,2,4,blue
2,2,3,5,blue


- get_dummies를 적용하여 범주형 데이터 color에 가변수 추가


In [9]:
edges.dtypes

source     int64
target     int64
weight     int64
color     object
dtype: object

In [10]:
pd.get_dummies(edges)

,source,target,weight,color_blue,color_red
0,0,2,3,False,True
1,1,2,4,True,False
2,2,3,5,True,False


- 범부형 데이터의 개수만큼 가변수(dummy variable)를 생성하여 존재 유무를 1 또는 0으로 표현


In [11]:
pd.get_dummies(edges["color"])

,blue,red
0,False,True
1,True,False
2,True,False


In [12]:
pd.get_dummies(edges[["color"]])

,color_blue,color_red
0,False,True
1,True,False
2,True,False


- 필요에 따라 정수형을 객체로 변경해서 처리
- weight는 숫자로 되어 있지만 기수형 데이터
- 데이터를 M, L, XL로 변경하여 원핫 인코딩을 적용


- 데이터를 원핫 인코딩 형태로 변경한 후 필요에 따라 병합이나 연결로 두 가지의 데이터를 합침


In [13]:
weight_dict = {3: "M", 4: "L", 5: "XL"}
edges["weight_sign"] = edges["weight"].map(weight_dict)
weight_sign = pd.get_dummies(edges["weight_sign"])
weight_sign

,L,M,XL
0,False,True,False
1,True,False,False
2,False,False,True


In [14]:
pd.concat([edges, weight_sign], axis=1)

,source,target,weight,color,weight_sign,L,M,XL
0,0,2,3,red,M,False,True,False
1,1,2,4,blue,L,True,False,False
2,2,3,5,blue,XL,False,False,True
